In [7]:
import ipywidgets as widgets
from IPython.display import display
import datetime, json, uuid

# ============================================================
# PRO115 – Aviso de Falla en Instalaciones
# HMI oficial (BASE visual estilo PRO173/PRO174)
# ============================================================

EN_CURSO = "EN_CURSO"
BLOQUEADO = "BLOQUEADO"
DETENIDO_STOP = "DETENIDO_STOP"
FINALIZADO = "FINALIZADO"

def _now_iso():
    return datetime.datetime.now().isoformat(timespec="seconds")

# ------------------------------------------------------------
# ZONA EDITABLE (cambia textos / checklist / validaciones aquí)
# ------------------------------------------------------------

def _es_opcional_por_si_aplica(texto: str) -> bool:
    """Si el texto contiene 'si aplica' en cualquier posición, no es obligatorio."""
    return "si aplica" in (texto or "").lower()

# ✅ Checklist de seguridad (se mantiene en todos los pasos)
CHECK_SEGURIDAD = "Declaro que NO existe una condición que afecta la seguridad"

# Motivos STOP (puedes ajustar)
MOTIVOS_STOP = [
    "Condición insegura detectada",
    "Falta de información crítica",
    "Sistema no disponible (SIGO / Mante / correo)",
    "No se logra contacto con COC/CDC",
    "Otro",
]

# NODOS: flujo 1→2→3→4→5→6→(prueba unidad?)→7→8→9→END_OK
# En el nodo de decisión, si selecciona RECHAZA/NO AUTORIZA => END_RECHAZO
NODOS = {
    "T1": {
        "type": "task",
        "titulo": "1 - Informar telefónicamente",
        "rol": "Operador SCC",
        "descripcion": (
            "El Operador Sala Control Central (SCC) informa vía telefónica en forma inmediata al Operador de Centro "
            "de Operaciones de Colbún en Santiago (COC) las causas que provocaron la limitación o falla de la "
            "instalación.\n\n"
            "Si no cuenta con toda la información, deberá aportar la información que ha tomado conocimiento y "
            "complementarla una vez conozca los demás detalles."
        ),
        "acciones": [
            "Informar vía telefónica al Operador COC las causas que provocaron la limitación o falla.",
            "Si no cuenta con toda la información, aportar la información conocida y complementarla posteriormente.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Informar vía telefónica al Operador COC las causas que provocaron la limitación o falla.",
            "Si no cuenta con toda la información, aportar la información conocida y complementarla posteriormente.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se informó telefónicamente al COC?",
        "next": "T2",
    },
    "T2": {
        "type": "task",
        "titulo": "2 - Informar a CDEC-SIC",
        "rol": "Operador COC",
        "descripcion": "Informa vía telefónica al Despachador CDC de la falla ocurrida.",
        "acciones": [
            "Informar vía telefónica al Despachador CDC de la falla ocurrida.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Informar vía telefónica al Despachador CDC de la falla ocurrida.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se informó al CDEC-SIC (CDC)?",
        "next": "T3",
    },
    "T3": {
        "type": "task",
        "titulo": "3 - Registrar Aviso de Falla",
        "rol": "Operador SCC",
        "descripcion": (
            "a) Ingresa un 'aviso de falla' en Sistema SIGO, el que a su vez envía en forma automática un correo "
            "electrónico dirigido al Centro de Operaciones de Colbún en Santiago, con copia a la lista de distribución "
            "definida para cada Central.\n\n"
            "En caso que SIGO no está disponible, completa formulario y lo envía a través de correo electrónico a la "
            "misma lista de distribución.\n\n"
            "b) Se comunica vía telefónica con el Operador COC para informar que ha ingresado el Aviso de Falla.\n\n"
            "Nota: La fecha y hora de inicio del aviso de falla queda establecido por el momento en que ocurre la falla.\n"
            "Plazo: A más tardar 1,45 horas desde ocurrido el evento de desconexión."
        ),
        "acciones": [
            "Ingresar un aviso de falla en SIGO (envío automático de correo a COC + distribución).",
            "Si aplica, Si SIGO no está disponible, completar formulario y enviarlo por correo a la misma distribución.",
            "Informar vía telefónica al Operador COC que se ingresó el Aviso de Falla.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Ingresar un aviso de falla en SIGO (envío automático de correo a COC + distribución).",
            "Si SIGO no está disponible, completar formulario y enviarlo por correo a la misma distribución.",
            "Informar vía telefónica al Operador COC que se ingresó el Aviso de Falla.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se registró el Aviso de Falla y se informó al COC?",
        "next": "T4",
    },
    "T4": {
        "type": "task",
        "titulo": "4 - Ingresar registro de falla en CDEC-SIC",
        "rol": "Operador COC",
        "descripcion": (
            "a) Revisa en SIGO (o formulario) que la información sea coherente y describa adecuadamente la situación. "
            "Si está incompleto, se comunica con SCC para complementar.\n\n"
            "b) Registra aviso de falla en Sistema Mante (web CDEC-SIC). Mante entrega un número correlativo.\n\n"
            "c) Informa al Despachador CDC el ingreso y el número correlativo obtenido. Luego recibe instrucciones.\n\n"
            "d) Informa al Operador SCC el ingreso en CDEC-SIC y el número con el cual quedó registrado."
        ),
        "acciones": [
            "Revisar en SIGO (o formulario) coherencia y completitud del aviso; si falta info, coordinar complemento con SCC.",
            "Registrar aviso de falla en Sistema Mante (web CDEC-SIC) y obtener número correlativo.",
            "Informar al Despachador CDC el ingreso y número correlativo; recibir instrucciones según condición SIC.",
            "Informar al Operador SCC el ingreso y número correlativo.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Revisar en SIGO (o formulario) coherencia y completitud del aviso; si falta info, coordinar complemento con SCC.",
            "Registrar aviso de falla en Sistema Mante (web CDEC-SIC) y obtener número correlativo.",
            "Informar al Despachador CDC el ingreso y número correlativo; recibir instrucciones según condición SIC.",
            "Informar al Operador SCC el ingreso y número correlativo.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se registró la falla en CDEC-SIC (Mante) y se informó a CDC/SCC?",
        "next": "T5",
    },
    "T5": {
        "type": "task",
        "titulo": "5 - Informar disponibilidad de Instalación",
        "rol": "Operador SCC",
        "descripcion": (
            "Una vez que la falla se ha solucionado en forma parcial o total, informa vía telefónica al Operador COC "
            "la solución parcial o total del problema que originó la falla y que permite dejar disponible la instalación afectada."
        ),
        "acciones": [
            "Informar vía telefónica al Operador COC la solución parcial o total y disponibilidad de la instalación.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Informar vía telefónica al Operador COC la solución parcial o total y disponibilidad de la instalación.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se informó disponibilidad de la instalación al COC?",
        "next": "T6",
    },
    "T6": {
        "type": "task",
        "titulo": "6 - Informar restablecimiento de Instalación a CDEC-SIC",
        "rol": "Operador COC",
        "descripcion": (
            "a) Informa vía telefónica al Despachador CDC la situación anterior y solicita el permiso correspondiente "
            "para proceder con maniobras asociadas. Las autorizaciones quedarán supeditadas a condiciones del SIC; "
            "pudiendo no autorizarse, retrasarse o anularse.\n\n"
            "b) Informa vía telefónica al Operador SCC la decisión de CDEC-SIC."
        ),
        "acciones": [
            "Informar vía telefónica al Despachador CDC el restablecimiento (parcial/total) y solicitar permiso para maniobras.",
            "Informar vía telefónica al Operador SCC la decisión de CDEC-SIC.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Informar vía telefónica al Despachador CDC el restablecimiento (parcial/total) y solicitar permiso para maniobras.",
            "Informar vía telefónica al Operador SCC la decisión de CDEC-SIC.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se informó restablecimiento al CDC y se comunicó la decisión a SCC?",
        "next": "D_PRUEBA_UNIDAD",
    },
    "D_PRUEBA_UNIDAD": {
        "type": "decision",
        "titulo": "Decisión CDEC-SIC",
        "rol": "Despachador CDC / Operador COC",
        "descripcion": "Define si se debe realizar prueba de unidad. Si el CDC NO autoriza/RECHAZA, el proceso termina.",
        "pregunta": "¿Se debe realizar prueba de unidad?",
        "opciones": [
            {"label": "SÍ (se debe hacer prueba de unidad)", "next": "T7"},
            {"label": "NO (no se debe hacer prueba de unidad)", "next": "T9"},
            {"label": "RECHAZA / NO AUTORIZA", "next": "END_RECHAZO"},
        ],
        "ayuda": "Si el CDC no autoriza la maniobra, el procedimiento finaliza inmediatamente.",
    },
    "T7": {
        "type": "task",
        "titulo": "7 - Realizar periodo de prueba",
        "rol": "Operador SCC",
        "descripcion": "Sincroniza la unidad que presentaba la falla y monitorea su funcionamiento durante el periodo establecido por CDEC-SIC.",
        "acciones": [
            "Sincronizar la unidad que presentaba la falla.",
            "Monitorear el funcionamiento durante el periodo establecido por CDEC-SIC.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Sincronizar la unidad que presentaba la falla.",
            "Monitorear el funcionamiento durante el periodo establecido por CDEC-SIC.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se realizó el periodo de prueba según lo establecido por CDEC-SIC?",
        "next": "T8",
    },
    "T8": {
        "type": "task",
        "titulo": "8 - Coordinar fin de pruebas",
        "rol": "Operador COC",
        "descripcion": (
            "Contando con la información del estado de la instalación afectada por la falla y del resultado de la(s) "
            "prueba(s) realizada(s), se comunica con el Despachador del CDC para informar la situación final y coordinar "
            "fecha y hora de cancelación del aviso de falla."
        ),
        "acciones": [
            "Comunicar al Despachador CDC el estado final y resultados de la(s) prueba(s).",
            "Coordinar con el CDC la fecha y hora de cancelación del aviso de falla.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Comunicar al Despachador CDC el estado final y resultados de la(s) prueba(s).",
            "Coordinar con el CDC la fecha y hora de cancelación del aviso de falla.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se coordinó el fin de pruebas y cancelación del aviso de falla con el CDC?",
        "next": "T9",
    },
    "T9": {
        "type": "task",
        "titulo": "9 - Comunicar cierre de aviso de falla",
        "rol": "Operador COC",
        "descripcion": (
            "Se comunica vía telefónica con la SCC de la instalación afectada para informarle tanto de la cancelación "
            "del aviso de falla como del horario de cancelación."
        ),
        "acciones": [
            "Informar vía telefónica a la SCC la cancelación del aviso de falla y el horario de cancelación.",
            CHECK_SEGURIDAD,
        ],
        "checklist": [
            "Informar vía telefónica a la SCC la cancelación del aviso de falla y el horario de cancelación.",
            CHECK_SEGURIDAD,
        ],
        "validacion": "¿Se comunicó el cierre del aviso de falla a la SCC?",
        "next": "END_OK",
    },

    "END_OK": {
        "type": "end",
        "titulo": "🏁 Proceso finalizado",
        "rol": "HMI",
        "descripcion": "Se completaron los pasos del PRO115. Puede exportar el JSON auditable si lo requiere.",
        "mensaje": "Fin del procedimiento.",
        "estado_final": FINALIZADO,
        "show_tables": True,
    },
    "END_RECHAZO": {
        "type": "end",
        "titulo": "⛔ Proceso terminado por rechazo / no autorización",
        "rol": "HMI",
        "descripcion": "El CDC rechazó o no autorizó las maniobras. El procedimiento finaliza.",
        "mensaje": "Fin por rechazo/no autorización.",
        "estado_final": FINALIZADO,
        "show_tables": True,
    },
    "END_STOP": {
        "type": "end",
        "titulo": "🛑 Proceso detenido (STOP)",
        "rol": "HMI",
        "descripcion": "El proceso fue detenido por STOP. Revise los motivos y retome cuando corresponda.",
        "mensaje": "Fin por STOP.",
        "estado_final": DETENIDO_STOP,
        "show_tables": True,
    },
}

# ------------------------------------------------------------
# MOTOR HMI (mantener formato / UI estilo PRO173)
# ------------------------------------------------------------

class PRO115HMI:
    def __init__(self):
        self.nodo_id = "T1"
        self.estado = EN_CURSO
        self.run_id = str(uuid.uuid4())
        self.start_ts = _now_iso()
        self.end_ts = None

        self.inputs = {}
        self.decisiones = []
        self.logs = []
        self.historial = []

        self.output = widgets.Output()
        # Botones base PRO173
        self.btn_si = widgets.Button(description="SÍ", button_style="success", layout={"width":"32%","height":"44px"})
        self.btn_no = widgets.Button(description="NO", button_style="danger", layout={"width":"32%","height":"44px"})
        # ✅ STOP al lado de SÍ/NO
        self.btn_stop = widgets.Button(description="🛑 STOP", button_style="warning", layout={"width":"32%","height":"44px"})

        self.btn_volver = widgets.Button(description="⬅ Volver al paso anterior", layout={"width":"100%","height":"40px"})
        self.btn_exportar = widgets.Button(description="Exportar JSON (trazabilidad)", icon="download", layout={"width":"100%","height":"40px"})

        self.msg_box = widgets.HTML("")
        self._check_widgets = []
        self._decision_widget = None

        # Panel STOP (motivos)
        self.stop_panel = widgets.VBox([])
        self.sel_stop_motivos = widgets.SelectMultiple(options=MOTIVOS_STOP, layout=widgets.Layout(width="100%", height="120px"))
        self.txt_stop_detalle = widgets.Textarea(placeholder="Detalle (opcional)", layout=widgets.Layout(width="100%", height="70px"))
        self.btn_confirm_stop = widgets.Button(description="Confirmar STOP", button_style="warning", layout={"width":"100%","height":"40px"})
        self.btn_cancel_stop = widgets.Button(description="Cancelar STOP", layout={"width":"100%","height":"40px"})
        self._stop_open = False

        # Wire
        self.btn_si.on_click(self._on_si)
        self.btn_no.on_click(self._on_no)
        self.btn_stop.on_click(self._on_stop_open)
        self.btn_confirm_stop.on_click(self._on_stop_confirm)
        self.btn_cancel_stop.on_click(self._on_stop_cancel)
        self.btn_volver.on_click(self._on_volver)
        self.btn_exportar.on_click(self._on_exportar)

        display(self.output)
        self.iniciar()

    def _log(self, tipo, data=None):
        self.logs.append({
            "ts": _now_iso(),
            "tipo": tipo,
            "nodo": self.nodo_id,
            "data": data or {}
        })

    def _push_history(self):
        self.historial.append(self.nodo_id)

    def _pop_history(self):
        if self.historial:
            return self.historial.pop()
        return None

    def _clear_msg(self):
        self.msg_box.value = ""

    def _msg(self, text, kind="warn"):
        if kind == "ok":
            self.msg_box.value = f"<div style='margin-top:10px;padding:12px;border-radius:10px;background:#dcfce7;border:1px solid #22c55e;color:#14532d;'><b>{text}</b></div>"
        else:
            self.msg_box.value = f"<div style='margin-top:10px;padding:12px;border-radius:10px;background:#fee2e2;border:1px solid #ef4444;color:#7f1d1d;'><b>{text}</b></div>"

    def _render_header(self, n):
        badge = f"<span style='display:inline-block;padding:4px 10px;border-radius:999px;background:#eef2ff;border:1px solid #c7d2fe;font-size:12px;color:black;'><b>ROL:</b> {n.get('rol','')}</span>"
        return widgets.HTML(f"""
        <div style="padding:16px;border-radius:12px;background:#f8fafc;border:1px solid #e2e8f0;">
            <div style="font-size:12px;color:#0f172a;"><b>PRO115</b> – Aviso de Falla en Instalaciones</div>
            <div style="margin-top:6px;font-size:22px;color:#0f172a;"><b>{n.get('titulo','')}</b></div>
            <div style="margin-top:8px;">{badge}</div>
            <div style="margin-top:10px;color:#0f172a;font-size:14px;line-height:1.35;white-space:pre-wrap;">{n.get('descripcion','')}​</div>
        </div>
        """)

    def _render_task(self, n):
        valid = n.get("validacion","")

        self._check_widgets = [
            widgets.Checkbox(description=item, value=False, layout=widgets.Layout(width="100%"))
            for item in (n.get("checklist",[]) or [])
        ]

        checklist_box = widgets.VBox([])
        if self._check_widgets:
            checklist_box = widgets.VBox([
                widgets.HTML("""
                <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:13px;color:#0f172a;"><b>🧾 ACCIONES (Checklist)</b></div>
                    <div style="margin-top:8px;font-size:12px;color:#0f172a;opacity:0.9;">
                        Ítems marcados como <b>“Si aplica”</b> no son obligatorios para avanzar.
                    </div>
                </div>
                """),
                widgets.VBox(self._check_widgets)
            ])

        valid_box = widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:2px solid #0ea5e9;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>✅ ¡VALIDACIÓN!</b></div>
                <div style="margin-top:8px;font-size:16px;color:#0f172a;"><b>{valid}</b></div>
                <div style="margin-top:6px;font-size:12px;color:#0f172a;">Confirma con <b>SÍ</b> para avanzar. Si respondes <b>NO</b>, el paso queda bloqueado.</div>
            </div>
        """)

        return widgets.VBox([checklist_box, valid_box])

    def _render_decision(self, n):
        opts = n.get("opciones",[])
        radios = widgets.RadioButtons(
            options=[(o["label"], o["next"]) for o in opts],
            layout={"width":"100%"},
            style={"description_width":"initial"},
        )
        help_txt = n.get("ayuda","")
        help_html = f"<div style='margin-top:10px;font-size:12px;color:#0f172a;opacity:0.9;'><b>Nota:</b> {help_txt}</div>" if help_txt else ""
        return widgets.VBox([
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>🔶 DECISIÓN (rombo)</b></div>
                <div style="margin-top:8px;font-size:16px;color:#0f172a;"><b>{n.get('pregunta','')}</b></div>
                {help_html}
            </div>
            """),
            radios
        ]), radios

    def _checklist_obligatorio_ok(self):
        oblig = []
        for cb in self._check_widgets:
            if not _es_opcional_por_si_aplica(cb.description):
                oblig.append(cb)
        return all(cb.value for cb in oblig) if oblig else True

    def _render_stop_panel(self):
        if not self._stop_open:
            self.stop_panel.children = []
            return
        self.stop_panel.children = [
            widgets.HTML("""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:2px solid #f59e0b;background:#fffbeb;">
                <div style="font-size:14px;color:#0f172a;"><b>🛑 STOP</b> — Seleccione motivo(s) y detalle (opcional).</div>
            </div>
            """),
            self.sel_stop_motivos,
            self.txt_stop_detalle,
            widgets.HBox([self.btn_confirm_stop, self.btn_cancel_stop], layout=widgets.Layout(gap="8px"))
        ]

    def _render_tables_if_end(self, n):
        if n.get("type") != "end" or not n.get("show_tables", False):
            return widgets.VBox([])
        inputs_rows = "".join([f"<tr><td><b>{k}</b></td><td>{(v or '')}</td></tr>" for k, v in self.inputs.items()])
        if not inputs_rows:
            inputs_rows = "<tr><td colspan='2'>(sin inputs)</td></tr>"

        dec_rows = "".join([f"<tr><td>{d.get('ts','')}</td><td>{d.get('nodo','')}</td><td>{d.get('seleccion','')}</td></tr>" for d in self.decisiones])
        if not dec_rows:
            dec_rows = "<tr><td colspan='3'>(sin decisiones)</td></tr>"

        return widgets.VBox([
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>📌 INPUTS</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:10px;">
                    <thead><tr><th style="border:1px solid #e2e8f0;padding:6px;text-align:left;">Campo</th>
                              <th style="border:1px solid #e2e8f0;padding:6px;text-align:left;">Valor</th></tr></thead>
                    <tbody>{inputs_rows}</tbody>
                </table>
            </div>
            """),
            widgets.HTML(f"""
            <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                <div style="font-size:13px;color:#0f172a;"><b>🧭 DECISIONES / EVENTOS</b></div>
                <table style="width:100%;border-collapse:collapse;margin-top:10px;">
                    <thead><tr><th style="border:1px solid #e2e8f0;padding:6px;text-align:left;">Timestamp</th>
                              <th style="border:1px solid #e2e8f0;padding:6px;text-align:left;">Nodo</th>
                              <th style="border:1px solid #e2e8f0;padding:6px;text-align:left;">Selección</th></tr></thead>
                    <tbody>{dec_rows}</tbody>
                </table>
            </div>
            """),
        ])

    def _render_footer(self):
        """Footer con controles (mismo patrón PRO173/PRO174)."""
        return widgets.VBox([
            widgets.HBox(
                [self.btn_si, self.btn_no, self.btn_stop],
                layout=widgets.Layout(justify_content="space-between", gap="8px", margin="10px 0")
            ),
            self.btn_volver,
            widgets.HTML("<div style='height:10px;'></div>"),
            self.btn_exportar,
            widgets.HTML("<div style='height:10px;'></div>"),
            self.stop_panel,
            self.msg_box,
        ])

    def _render(self):
        with self.output:
            self.output.clear_output()
            self._clear_msg()

            n = NODOS[self.nodo_id]
            header = self._render_header(n)

            if n["type"] == "task":
                body = self._render_task(n)
                self._decision_widget = None
            elif n["type"] == "decision":
                body, radios = self._render_decision(n)
                self._decision_widget = radios
                self._check_widgets = []
            else:
                body = widgets.HTML(f"""
                <div style="margin-top:12px;padding:14px;border-radius:12px;border:1px solid #e2e8f0;background:#ffffff;">
                    <div style="font-size:14px;color:#0f172a;"><b>{n.get('mensaje','')}</b></div>
                </div>
                """)
                self._decision_widget = None
                self._check_widgets = []

            self._render_stop_panel()
            end_tables = self._render_tables_if_end(n)
            footer = self._render_footer()

            display(widgets.VBox([header, body, end_tables, footer]))

    def iniciar(self):
        self._render()

    # ---------- eventos ----------
    def _on_si(self, _):
        n = NODOS[self.nodo_id]
        if n["type"] == "end":
            self._msg("Este es un nodo final.", "ok")
            return

        if n["type"] == "decision":
            if self._decision_widget is None or self._decision_widget.value is None:
                self._msg("Debe seleccionar una opción.", "warn")
                self._log("VALIDACION_FALLA", {"mensaje": "sin selección"})
                return
            # Registrar decisión: label
            label_map = dict(self._decision_widget.options)
            self.decisiones.append({
                "ts": _now_iso(),
                "nodo": self.nodo_id,
                "seleccion": label_map.get(self._decision_widget.value, str(self._decision_widget.value))
            })
            nxt = self._decision_widget.value
            self._push_history()
            self._log("AVANZA", {"next": nxt})
            self.nodo_id = nxt

            if NODOS[self.nodo_id]["type"] == "end":
                self.estado = NODOS[self.nodo_id].get("estado_final", FINALIZADO)
                self.end_ts = _now_iso()

            self._render()
            return

        # task: validar checklist obligatorio
        if not self._checklist_obligatorio_ok():
            self._msg("Debe completar las acciones obligatorias antes de avanzar.", "warn")
            self._log("VALIDACION_FALLA", {"mensaje": "checklist obligatorio incompleto"})
            return

        self.decisiones.append({"ts": _now_iso(), "nodo": self.nodo_id, "seleccion": "OK"})
        self._push_history()
        nxt = n.get("next")
        self._log("AVANZA", {"next": nxt})
        self.nodo_id = nxt

        if NODOS[self.nodo_id]["type"] == "end":
            self.estado = NODOS[self.nodo_id].get("estado_final", FINALIZADO)
            self.end_ts = _now_iso()

        self._render()

    def _on_no(self, _):
        n = NODOS[self.nodo_id]
        if n["type"] == "decision":
            self._msg("En decisiones, seleccione la opción en el rombo y luego presione SÍ.", "warn")
            return
        if n["type"] == "end":
            self._msg("Este es un nodo final.", "ok")
            return
        self.estado = BLOQUEADO
        self._log("BLOQUEA", {"motivo": "Respuesta NO en validación"})
        self._msg("Paso bloqueado: respondió NO en la validación.", "warn")
        self._render()

    def _on_volver(self, _):
        prev = self._pop_history()
        if prev is None:
            self._msg("No hay paso anterior.", "warn")
            return
        self.nodo_id = prev
        self.estado = EN_CURSO
        self._log("VOLVER", {"to": prev})
        self._render()

    def _on_stop_open(self, _):
        self._stop_open = True
        self._render()

    def _on_stop_cancel(self, _):
        self._stop_open = False
        self.sel_stop_motivos.value = ()
        self.txt_stop_detalle.value = ""
        self._render()

    def _on_stop_confirm(self, _):
        motivos = list(self.sel_stop_motivos.value)
        detalle = (self.txt_stop_detalle.value or "").strip()
        self.decisiones.append({"ts": _now_iso(), "nodo": self.nodo_id, "seleccion": f"STOP: {motivos} | {detalle}"})
        self._log("STOP", {"motivos": motivos, "detalle": detalle})
        self.estado = DETENIDO_STOP
        self.end_ts = _now_iso()
        self._push_history()
        self.nodo_id = "END_STOP"
        self._stop_open = False
        self._render()

    def _on_exportar(self, _):
        payload = {
            "proceso": "PRO115 – Aviso de Falla en Instalaciones",
            "run_id": self.run_id,
            "estado": self.estado,
            "start_ts": self.start_ts,
            "end_ts": self.end_ts,
            "current_node": self.nodo_id,
            "history_stack": list(self.historial),
            "decisiones": list(self.decisiones),
            "inputs": dict(self.inputs),
            "logs": list(self.logs),
            "export_ts": _now_iso(),
        }
        pretty = json.dumps(payload, ensure_ascii=False, indent=2)
        self.msg_box.value = f"""
        <div style='margin-top:10px;padding:12px;border-radius:10px;background:#dcfce7;border:1px solid #22c55e;color:#14532d;'>
            <b>📦 Export JSON (trazabilidad)</b>
            <pre style='white-space:pre-wrap;margin-top:10px;color:#14532d;'>{pretty}</pre>
        </div>
        """

# Ejecutar
hmi = PRO115HMI()


Output()

In [4]:
hmi._on_si(None)